In [27]:
# treshold ~ precision, recall, saved commit

# file = 'evaluation/multi_commits_rly_false_pairs_result.txt'
# file = 'evaluation/multi_commits_msr_result.txt'
# file = 'detection/multi_commits_second_nondup_part_newret.txt'

# file = 'detection/rly_false_pairs_newret.txt'
# file = 'detection/big_false_data_newret.txt'

# file = 'detection/msr_multi_commits_no_repeat_newret.txt'
# file = 'detection/msr_multi_commits_no_repeat_newret5.txt'
# file = 'detection/multi_commits_second_nondup_part_newret.txt'
# file = 'detection/multi_commits_second_nondup_v_newret.txt'

# file = 'detection/msr_multi_commits_okret.txt'
# file = 'detection/msr_multi_commits_okret_tds.txt'


lastc = {}
with open('detection/mulc_second_msr_pairs_lastc.txt') as f:
    for t in f.readlines():
        ts = t.strip().split(' : ')
        r, n1, n2 = ts[0].split()
        vs = ts[1][2:-2].split('), (')
        lastc[(r, n1, n2)] = []
        for v in vs:
            x, y = v.split(',')
            lastc[(r, n1, n2)].append((int(x),int(y)))

treshold = 0.6

import git

def check_overlap_c(p1, p2):
    for c1 in git.get_pull_commit(p1):
        for c2 in git.get_pull_commit(p2):
            if (len(c1['commit']['message']) > 0) and (c1['commit']['message'] == c2['commit']['message']): # repeat commit
                return True
    return False



def calc(file):
    with open(file) as f:
        ps = f.readlines()

    second_r = ['elastic/elasticsearch', 'cocos2d/cocos2d-x', 'emberjs/ember.js', 'django/django', 'rails/rails', 'angular/angular.js', 'dotnet/corefx', 'ceph/ceph', 'ansible/ansible', 'hashicorp/terraform', 'joomla/joomla-cms', 'facebook/react', 'JuliaLang/julia', 'docker/docker']

    save = []
    cases = []
    save_tot = 0
    save_times, save_times_p = 0, 0
    
    for p in ps:
        p1, hist = p.strip()[:-1].split('[')

        ds = p1.split()
        r, n1, n2 = ds[0], ds[1], ds[2]
        
        '''
        if check_overlap_c(git.get_pull(r, n1), git.get_pull(r, n2)):
            continue
        '''

        '''
        if r not in second_r:
            continue
        '''

        hist = [float(h) for h in hist.split(',')]

        if len(hist) == 1:
            continue

        if 'msr' in file:
            if (len(hist) != len(lastc[(r, n1, n2)])):
                raise Exception('len not equal!', r, n1, n2)
        
        s = -1
        for i in range(len(hist)):
            if hist[i] >= treshold:
                s = len(hist) - i - 1
                if 'msr' in file:
                    sx, sy = lastc[(r, n1, n2)][i][0], lastc[(r, n1, n2)][i][1]
                    save_tot += max(sx, sy)
                    if max(sx, sy) > 0:
                        save_times += 1
                        if min(sx, sy) == 0:
                            save_times_p += 1                    
                break

        save.append(s)
        cases.append([r, n1, n2])

        '''
        if s > 0:
            print(r, n1, n2)
        '''
    
    
    if 'msr' in file:
        # print(save_times_p, save_times, len(save))
        # print(1.0 * save_times_p / save_times if save_times > 0 else 'N/A')
        pass
    
    return save, cases, 1.0 * save_tot / len(save)

'''
def P(save):
    no_zero = 0
    for i in save:
        if i > 0:
            no_zero += 1
    return no_zero
    
    no_neg = 0
    for i in save:
        if i >= 0:
            no_neg += 1
'''


treshold = 0.48

for tres in range(3700, 7001, 25):
# for tres in range(6800, 7000, 25):
#for tres in range(4500, 4501, 1):
#for tres in range(5500, 5501, 1):
#for tres in range(6000, 6001, 1):

    treshold = 1.0 * tres / 10000

    file1 = 'detection/mulc_second_nondup_okret_tds.txt' # negative
    file2 = 'detection/mulc_second_msr_pairs_okret_tds.txt' #positive
    
    ret_n, ca_n, _ = calc(file1)
    fp = len(list(filter(lambda x: x >= 0, ret_n)))
    
    repos = set([x[0] for x in ca_n])
    
    ret_p, ca_p, avg_save = calc(file2)
    tp = len(list(filter(lambda x: x >= 0, ret_p)))

    precision = tp / (tp + fp) if tp + fp > 0 else 1.0
    recall = tp / len(ret_p)
    # saved_c = 1.0 * sum(list(filter(lambda x: x > 0, ret_p))) / 2 / len(ret_p)
    
    FPR = fp / len(ret_n)
    
    #print('treshold', treshold)
    #print('precision', precision)
    #print('recall', recall)
    #print('saved_c', saved_c)
    
    # print(treshold, fp, sep='\t')
    print(treshold, precision, recall, avg_save, sep='\t')
    
    '''
    precision_num = {}
    recall_num = {}
    
    for r in repos:
        precision_num[r] = [0, 0]
        recall_num[r] = [0, 0]

    for i in range(len(ret_n)):
        r = ca_n[i][0]
        
        if ret_n[i] > 0:
            precision_num[r][1] += 1
    
    
    for i in range(len(ret_p)):
        r = ca_p[i][0]

        recall_num[r][1] += 1

        if ret_p[i] > 0:
            precision_num[r][0] += 1
            precision_num[r][1] += 1
            
            recall_num[r][0] += 1
    
    def safe_div(n, d):
        return n / d if d > 0 else 0
    
    pre_tot, pre_num = 0, 0
    rec_tot, rec_num = 0, 0
    
    
    for r in repos:
        if precision_num[r][1] == 0:
            precision = -1
        else:
            precision = 1.0 * precision_num[r][0] / precision_num[r][1]
            pre_tot += precision
            pre_num += 1
            
        if recall_num[r][1] == 0:
            recall = -1
        else:
            recall = 1.0 * recall_num[r][0] / recall_num[r][1]
            rec_tot += recall
            rec_num += 1
        
        print('{:20}'.format(r), \
            precision_num[r][0], precision_num[r][1],\
            '{:.0%}'.format(precision), \
            recall_num[r][0], recall_num[r][1],\
            '{:.0%}'.format(recall), \
            #'%d%d' % (precision_num[r][0], precision_num[r][1]),\
            #'%d/%d' % (recall_num[r][0], recall_num[r][1]),\
            sep='\t')
    
    print(1.0 * pre_tot / pre_num)
    print(1.0 * rec_tot / rec_num)
    '''

'''
file1 = 'detection/mulc_second_nondup_okret_tds.txt.fil' #negative
file2 = 'detection/mulc_second_msr_pairs_okret_tds.txt.fil' #positive

print(len(calc(file1)))
print(len(calc(file2)))
'''
    

0.37	0.029623175778697452	1.0	5.904411764705882
0.3725	0.029623175778697452	1.0	5.904411764705882
0.375	0.029861670204201128	1.0	5.901960784313726
0.3775	0.030610709987966306	0.9975490196078431	5.8995098039215685
0.38	0.030654515327257662	0.9975490196078431	5.8995098039215685
0.3825	0.030905915407396158	0.9975490196078431	5.897058823529412
0.385	0.031028436380269878	0.9975490196078431	5.897058823529412
0.3875	0.03149179820489013	0.9975490196078431	5.894607843137255
0.39	0.03167068710606179	0.9975490196078431	5.894607843137255
0.3925	0.031911557158538495	0.9975490196078431	5.879901960784314
0.395	0.03194662480376766	0.9975490196078431	5.879901960784314
0.3975	0.032057340894770005	0.9975490196078431	5.879901960784314
0.4	0.032057340894770005	0.9975490196078431	5.879901960784314
0.4025	0.03205986608901142	0.9975490196078431	5.879901960784314
0.405	0.032062391681109186	0.9975490196078431	5.879901960784314
0.4075	0.032062391681109186	0.9975490196078431	5.879901960784314
0.41	0.0320674440592

"\nfile1 = 'detection/mulc_second_nondup_okret_tds.txt.fil' #negative\nfile2 = 'detection/mulc_second_msr_pairs_okret_tds.txt.fil' #positive\n\nprint(len(calc(file1)))\nprint(len(calc(file2)))\n"

In [1]:
r2 = set()
with open('data/clf/second_msr_pairs_nolarge.txt') as f:
	for t in f.readlines():
		r, n1, n2 = t.split()
		r2.add(r)

print(r2)


{'angular/angular.js', 'facebook/react', 'rails/rails', 'hashicorp/terraform', 'joomla/joomla-cms', 'docker/docker', 'ceph/ceph', 'dotnet/corefx', 'django/django', 'cocos2d/cocos2d-x', 'ansible/ansible', 'JuliaLang/julia', 'elastic/elasticsearch', 'emberjs/ember.js'}


In [21]:
from git import *

repo = 'rails/rails'
file = 'evaluation/rails_rails_select.txt'
with open(file) as f:
    for n in f.readlines():
        p = get_pull(repo, n.strip())
        if check_large(p):
            print(n)


In [42]:
treshold = 0.51

# file = 'evaluation/multi_commits_rly_false_pairs_result.txt'
# file = 'evaluation/multi_commits_msr_result.txt'

file = 'detection/multi_commits_second_nondup_part_newret.txt'

# file = 'detection/rly_false_pairs_newret.txt'
# file = 'detection/msr_multi_commits_no_repeat_newret.txt'
# file = 'detection/big_false_data_newret.txt'

with open(file) as f:
    ps = f.readlines()

# second_r = ['elastic/elasticsearch', 'cocos2d/cocos2d-x', 'emberjs/ember.js', 'django/django', 'rails/rails', 'angular/angular.js', 'dotnet/corefx', 'ceph/ceph', 'ansible/ansible', 'hashicorp/terraform', 'joomla/joomla-cms', 'facebook/react', 'JuliaLang/julia', 'docker/docker']

save = []
for p in ps:
    p1, hist = p.strip()[:-1].split('[')
    r, n1, n2, ori, det, delta, zz = p1.split()
    
    '''
    if r not in second_r:
        continue
    '''
    
    if det == '2':
        continue

    hist = [float(h) for h in hist.split(',')]
        
    
    if len(hist) == 1:
        continue
    

    s = -1
    for i in range(len(hist)):
        if hist[i] >= treshold:
            s = len(hist) - i - 1
            break
    save.append(s)
    
    
    if s >= 0:
        print(r, n1, n2, len(hist))
    
    
print('all', len(save))

no_fuyi = 0
for i in save:
    if i >= 0:
        no_fuyi += 1

print('no_zero=', no_fuyi / len(save))


angular/angular.js 15836 5385 2
all 279
no_zero= 0.0035842293906810036


In [47]:
import git
from datetime import datetime, timedelta


def checkt(repo, num1, num2):    
    p1 = git.get_pull(repo, num1)
    p2 = git.get_pull(repo, num2)
    
    all_pa = git.get_pull_commit(p1)
    all_pb = git.get_pull_commit(p2)
    
    l_a, l_b = len(all_pa), len(all_pb)
    num_a, num_b = 0, 0
    now_a, now_b = None, None
    
    numt = 0
    while True:
        pa = all_pa[num_a] if num_a < l_a else None
        pb = all_pb[num_b] if num_b < l_b else None
        if (pa is None) and (pb is None):
            break
        
        if pa:
            pat = datetime.strptime(pa['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
        if pb:
            pbt = datetime.strptime(pb['commit']['author']['date'], "%Y-%m-%dT%H:%M:%SZ")
        
        if (pb is None) or (pa and (pat < pbt)):
            num_a += 1
            now_a = pa
        else:
            num_b += 1
            now_b = pb
        
        if now_a and now_b:
            if (num_a < l_a) or (num_b < l_b):
                numt += 1
    
    if numt > 0:
        return True
    else:
        return False

num, tot = 0, 0
with open('data/clf/second_msr_pairs_nolarge.txt') as f:
    for t in f.readlines():
        r, n1, n2 = t.split()
        
        tot += 1
        if checkt(r, n1, n2):
            num += 1
        
        if tot % 100 == 0:
            print(1.0 * num / tot)

print(num)
print(1.0 * num / tot)


0.27
0.25
0.2966666666666667
0.34
0.324
0.31833333333333336
0.3142857142857143
0.3425
0.3511111111111111
0.327
333
0.31563981042654027


In [5]:
import git

def fil(file):
    with open(file) as f:
        ps = f.readlines()
    
    cnt = 0
    for p in ps:
        p1, hist = p.strip()[:-1].split('[')
        
        cnt += 1
        if cnt % 1000 == 0:
            print(cnt)
        
        ds = p1.split()
        r, n1, n2 = ds[0], ds[1], ds[2]

        if not check_overlap_c(git.get_pull(r, n1), git.get_pull(r, n2)):
            with open(file + '.fil', 'a') as f:
                print(p.strip(), file=f)

file1 = 'detection/mulc_second_nondup_okret_tds.txt.fil' #negative
file2 = 'detection/mulc_second_msr_pairs_okret_tds.txt.fil' #positive


fil(file2)
fil(file1)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000


In [2]:
# treshold ~ precision, recall, saved commit

file = 'tmp/ret_f.txt'


def calc(file):
    with open(file) as f:
        ps = f.readlines()

    save = []
    cases = []
    for p in ps:
        p1, hist = p.strip()[:-1].split('[')

        ds = p1.split()
        r, n1, n2 = ds[0], ds[1], ds[2]

        hist = [float(h) for h in hist.split(',')]

        if len(hist) == 1:
            continue


        s = -1
        for i in range(len(hist)):
            if hist[i] >= treshold:
                s = len(hist) - i - 1
                break
        save.append(s)
        cases.append([r, n1, n2])

    
    return save, cases

treshold = 0.6

ret_p, ca_p = calc(file)
tp = len(list(filter(lambda x: x >= 0, ret_p)))
saved_c = sum(list(filter(lambda x: x > 0, ret_p))) / len(ret_p)

print(saved_c)

'''
for tres in range(3700, 6801, 25):

    treshold = 1.0 * tres / 10000

    #file1 = 'detection/mulc_second_nondup_okret_tds.txt' # negative
    #file2 = 'detection/mulc_second_msr_pairs_okret_tds.txt' #positive

    ret_n, ca_n = calc(file1)
    fp = len(list(filter(lambda x: x >= 0, ret_n)))
    
    repos = set([x[0] for x in ca_n])
    
    ret_p, ca_p = calc(file2)
    tp = len(list(filter(lambda x: x >= 0, ret_p)))

    precision = tp / (tp + fp)
    recall = tp / len(ret_p)
    saved_c = sum(list(filter(lambda x: x > 0, ret_p))) / len(ret_p)
    
    FPR = fp / len(ret_n)
    
    print(treshold, precision, recall, saved_c, sep='\t')
'''
    

3.013888888888889


"\nfor tres in range(3700, 6801, 25):\n\n    treshold = 1.0 * tres / 10000\n\n    #file1 = 'detection/mulc_second_nondup_okret_tds.txt' # negative\n    #file2 = 'detection/mulc_second_msr_pairs_okret_tds.txt' #positive\n\n    ret_n, ca_n = calc(file1)\n    fp = len(list(filter(lambda x: x >= 0, ret_n)))\n    \n    repos = set([x[0] for x in ca_n])\n    \n    ret_p, ca_p = calc(file2)\n    tp = len(list(filter(lambda x: x >= 0, ret_p)))\n\n    precision = tp / (tp + fp)\n    recall = tp / len(ret_p)\n    saved_c = sum(list(filter(lambda x: x > 0, ret_p))) / len(ret_p)\n    \n    FPR = fp / len(ret_n)\n    \n    print(treshold, precision, recall, saved_c, sep='\t')\n"

In [3]:
def calc(file):
    with open(file) as f:
        ps = f.readlines()

    second_r = ['elastic/elasticsearch', 'cocos2d/cocos2d-x', 'emberjs/ember.js', 'django/django', 'rails/rails', 'angular/angular.js', 'dotnet/corefx', 'ceph/ceph', 'ansible/ansible', 'hashicorp/terraform', 'joomla/joomla-cms', 'facebook/react', 'JuliaLang/julia', 'docker/docker']

    save = []
    cases = []
    for p in ps:
        p1, hist = p.strip()[:-1].split('[')

        ds = p1.split()
        r, n1, n2 = ds[0], ds[1], ds[2]
        
        '''
        if check_overlap_c(git.get_pull(r, n1), git.get_pull(r, n2)):
            continue
        '''

        '''
        if r not in second_r:
            continue
        '''

        hist = [float(h) for h in hist.split(',')]

        if len(hist) == 1:
            continue


        s = -1
        for i in range(len(hist)):
            if hist[i] >= treshold:
                s = len(hist) - i - 1
                break
        save.append(s)
        cases.append([r, n1, n2])

        '''
        if s > 0:
            print(r, n1, n2)
        '''
    
    return save, cases


for tres in range(3700, 7001, 25):

    treshold = 1.0 * tres / 10000
    
    
    file2 = 'detection/mulc_second_msr_pairs_okret_tds.txt' #positive

    ret_p, ca_p = calc(file2)
    tp1 = len(list(filter(lambda x: x > 0, ret_p)))
    tp2 = len(list(filter(lambda x: x >= 0, ret_p)))
    
    '''
    total = 1149
    # print(treshold, tp1, tp2, len(ret_p), sep='\t')
    print(treshold, tp1, tp2 - tp1, len(ret_p) - tp2, total - len(ret_p), sep='\t')
    '''
    
    file1 = 'detection/mulc_second_nondup_okret_tds.txt' # negative
    ret_n, ca_n = calc(file1)
    fp1 = len(list(filter(lambda x: x > 0, ret_n)))
    fp2 = len(list(filter(lambda x: x >= 0, ret_n)))
    
    print(treshold, tp1, tp2 - tp1, fp2, sep='\t')
    
    

0.37	408	0	13365
0.3725	408	0	13365
0.375	407	1	13255
0.3775	406	1	12889
0.38	406	1	12870
0.3825	406	1	12762
0.385	406	1	12710
0.3875	406	1	12517
0.39	406	1	12444
0.3925	406	1	12347
0.395	406	1	12333
0.3975	406	1	12289
0.4	406	1	12289
0.4025	406	1	12288
0.405	406	1	12287
0.4075	406	1	12287
0.41	406	1	12285
0.4125	405	2	11459
0.415	402	4	8019
0.4175	402	4	7472
0.42	402	4	6646
0.4225	395	7	3728
0.425	384	12	1296
0.4275	382	13	1153
0.43	380	14	1057
0.4325	379	14	1018
0.435	378	14	935
0.4375	377	15	917
0.44	376	15	905
0.4425	375	16	882
0.445	375	16	881
0.4475	370	19	689
0.45	368	19	656
0.4525	367	17	582
0.455	367	17	525
0.4575	361	18	470
0.46	357	20	417
0.4625	357	20	358
0.465	355	20	310
0.4675	352	21	294
0.47	345	24	276
0.4725	344	24	268
0.475	342	24	260
0.4775	341	25	252
0.48	339	24	240
0.4825	336	22	232
0.485	335	22	229
0.4875	331	23	206
0.49	328	24	193
0.4925	324	25	181
0.495	321	25	165
0.4975	318	26	151
0.5	306	26	142
0.5025	301	26	132
0.505	292	26	110
0.5075	287	28	98
0.51	281	26	91
